In [8]:
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

In [9]:
work_dir = '/home/nlp/achimoa/workspace/hebrew_text_retrieval/'
os.chdir(work_dir)

src_dir = os.path.join(work_dir, 'src')
if sys.path[0] not in src_dir:
    sys.path.insert(0, src_dir)

In [53]:
split = 'validation'  # Change to 'validation' if needed
dataset_name = 'heq'
df_queries = pd.read_csv(f'outputs/translation/{dataset_name}/{split}/gemini-2.0-flash-lite/queries_translated.csv')
df_documents = pd.read_csv(f'outputs/translation/{dataset_name}/{split}/gemini-2.0-flash-lite/documents_translated.csv')

In [54]:
print(list(df_queries.keys()))
print(df_queries.shape)
print(list(df_documents.keys()))
print(df_documents.shape)

['anchor_text', 'positive_text', 'index', 'paragraph_index', 'question_id', 'question', 'answer', 'context', 'context_hash', 'id', 'text', 'context_id', 'context_text', 'dataset_name', 'tokenizer', 'source_lang', 'target_lang', 'text_key', 'translation_key', 'context_key', 'translation_query_key', 'translation_document_key', 'system_prompt', 'user_prompt', 'prompt_prefix', 'prompt_file_name', 'translation', 'input_tokens', 'output_tokens', 'model_name', 'model_time', 'translation_time', 'timestamp', 'batch_idx', 'batch_size', 'batch_datetime', 'translation_datetime']
(168, 37)
['anchor_text', 'positive_text', 'index', 'paragraph_index', 'question_id', 'question', 'answer', 'context', 'context_hash', 'id', 'text', 'segment_id', 'segment_text', 'dataset_name', 'tokenizer', 'source_lang', 'target_lang', 'text_key', 'translation_key', 'context_key', 'translation_query_key', 'translation_document_key', 'system_prompt', 'user_prompt', 'prompt_prefix', 'prompt_file_name', 'translation', 'inpu

In [55]:
# Get the set of GUIDs from df_documents
document_guids = set(df_documents['context_hash'])

# Check which df_queries['guid'] values exist in df_documents
mask = df_queries['context_hash'].isin(document_guids)

# Count the number of matching records
matching_count = mask.sum()

print(f"Number of matching GUIDs: {matching_count}")


Number of matching GUIDs: 168


In [56]:
df_documents[['translation']].head()

,translation
0,"Yehuda ben Gerim told of his words, and as a r..."
1,"In his youth, Rashbi was among the sages who g..."
2,Mushroom motifs were found in the ruins of a M...
3,"However, there is evidence that heavy cannabis..."
4,There is a significant correlation between can...


In [57]:
df_documents[['id', 'context_hash', 'text', 'segment_id']]

,id,context_hash,text,segment_id
0,dc04e43cf20ca319f608b77dffabe637,dc04e43cf20ca319f608b77dffabe637,"יהודה בן גרים סיפר על דבריו, ובעקבות זאת נדון ...",0
1,8f9843fd2f675169908bac717c401891,8f9843fd2f675169908bac717c401891,"בצעירותו נמנה רשב""י עם החכמים שהתקבצו ב'כרם בי...",0
2,30696fe6b2459161b8f70051b1fc73d6,30696fe6b2459161b8f70051b1fc73d6,מוטיבים של פטריות נמצאו בהריסות של מקדש שבט המ...,0
3,68c786be386f4ccf1cb7f369b472eda4,68c786be386f4ccf1cb7f369b472eda4,"עם זאת, יש ראיות לכך שמעשני קנאביס כבדים עלולי...",0
4,91df80e716f986f70e2d9df9ff9c850c,91df80e716f986f70e2d9df9ff9c850c,קיים מתאם מובהק בין שימוש בקנאביס לסיכוי של אד...,0
...,...,...,...,...
163,08458c05979bcb17cfd028dc796700f3,08458c05979bcb17cfd028dc796700f3,"על המתחרה הגדולה, ענקית הקמעונאות האמריקאית אמ...",0
164,fc5ded771e6b6cb229f45974cac144c1,fc5ded771e6b6cb229f45974cac144c1,נראה כי iOS 13.1 כוללת בתוכה מספר פיצ׳רים נחמד...,0
165,d06bc63a8580aa09e56f66c1c99dfd71,d06bc63a8580aa09e56f66c1c99dfd71,לפני כשנה וחצי הודיעה אפל על שינויים במדיניות ...,0
166,554f2cc15435bab82125916f140b6eb7,554f2cc15435bab82125916f140b6eb7,גולת הכותרת של מכשירי הפיקסל 3 הקיימים היא כמו...,0


In [58]:
source_lang = 'hebrew'
target_lang = 'english'

df_documents2 = df_documents.groupby(['id', 'context_hash', 'text'], as_index=False).agg({
    'translation': lambda texts: ' '.join([str(t) for t in texts])
})

# Step 2: Rename columns as needed
df_documents2 = df_documents2.rename(columns={
    'text': f'context_{source_lang}',
    'translation': f'context_{target_lang}'
})

df_documents2['_source'] = 'heq'
print(df_documents2.columns)

# Step 3: Select required columns
df_documents2 = df_documents2[['id', 'context_hash', f'context_{source_lang}', f'context_{target_lang}', '_source']]

# Step 4: Save the DataFrame to a JSON file
output_file = f'data/{dataset_name}/{split}/documents.jsonl'
Path(output_file).parent.mkdir(parents=True, exist_ok=True)
df_documents2.to_json(output_file, orient='records', lines=True, force_ascii=False)

Index(['id', 'context_hash', 'context_hebrew', 'context_english', '_source'], dtype='object')


In [59]:
df_documents2.head(100)

,id,context_hash,context_hebrew,context_english,_source
0,0052c8418a1219c177f8c58eba5c62d2,0052c8418a1219c177f8c58eba5c62d2,תהליך הבדיקה די פשוט ומסתכם בהרמה של המכשיר מע...,The testing process is quite simple and involv...,heq
1,007d5f8170359f997f4e11fa828f10bf,007d5f8170359f997f4e11fa828f10bf,המודל הראשון שהציג מארק צוקרברג הוא מודל התאמת...,The first model presented by Mark Zuckerberg i...,heq
2,00b16524e9ffb4c53a4aadc4cbdf8253,00b16524e9ffb4c53a4aadc4cbdf8253,"עד שמזוודות שעוקבות אחרינו יהיו עניין שגרתי, מ...",Until suitcases that follow us become commonpl...,heq
3,01b754d25abd96aaf36da6e359a9d53e,01b754d25abd96aaf36da6e359a9d53e,למרות שבישראל רובינו משתמשים בוואטסאפ כדי לתקש...,Although in Israel most of us use WhatsApp to ...,heq
4,02dcea824fe833cb18fac79382796b5c,02dcea824fe833cb18fac79382796b5c,"בצד ההתפתחות הכלכלית, לברזיל מספר אתגרים כלכלי...","Alongside economic development, Brazil faces s...",heq
...,...,...,...,...,...
95,a6a4cc52d0c44ef74ee6795b1e7d021e,a6a4cc52d0c44ef74ee6795b1e7d021e,"יאיר לפיד, יו""ר יש עתיד, בוחר את נציגיו ברשימה...","Yair Lapid, chairman of Yesh Atid, chooses his...",heq
96,a7a4f386812a4ffbd6246bb54306f164,a7a4f386812a4ffbd6246bb54306f164,ערביי ארץ ישראל הושפעו גם מן ההתפתחויות המדיני...,The Arabs of Eretz Israel were also influenced...,heq
97,a88ad9e49f567211c84a47efc2b061de,a88ad9e49f567211c84a47efc2b061de,"הפיצ׳ר החדש של האפליקציה, שקיבל את השם התכליתי...","The new feature of the app, which received the...",heq
98,ae1891764843ff37238e39ef3a91194b,ae1891764843ff37238e39ef3a91194b,בשנת 2011 יצא לאור בהוצאת NMC אלבום האולפן שלו...,"In 2011, his studio album ""She Falls as if"", w...",heq


In [60]:
df_documents2.shape

(168, 5)

In [61]:
df_queries.head()

,anchor_text,positive_text,index,paragraph_index,question_id,question,answer,context,context_hash,id,...,input_tokens,output_tokens,model_name,model_time,translation_time,timestamp,batch_idx,batch_size,batch_datetime,translation_datetime
0,"מה אכלו הרשב""י ובנו במערה?","יהודה בן גרים סיפר על דבריו, ובעקבות זאת נדון ...",0,0,425478ad-1fb3-4a1a-a100-230cc56e2ccf,"מה אכלו הרשב""י ובנו במערה?",חרוב,"יהודה בן גרים סיפר על דבריו, ובעקבות זאת נדון ...",dc04e43cf20ca319f608b77dffabe637,425478ad-1fb3-4a1a-a100-230cc56e2ccf,...,413,40,gemini-2.0-flash-lite,1.645951,1.645968,2025-07-16 13:27:33.597191,0,1,2025-07-16 13:27:31.951201,2025-07-16 13:27:33.597200
1,למה הודח רבן גמליאל מנשיאותו?,"בצעירותו נמנה רשב""י עם החכמים שהתקבצו ב'כרם בי...",0,1,1139748b-3e29-4c9f-8ff8-180d2e734757,למה הודח רבן גמליאל מנשיאותו?,בעקבות יחסו לרבי יהושע,"בצעירותו נמנה רשב""י עם החכמים שהתקבצו ב'כרם בי...",8f9843fd2f675169908bac717c401891,1139748b-3e29-4c9f-8ff8-180d2e734757,...,335,41,gemini-2.0-flash-lite,1.770253,1.770270,2025-07-16 13:27:33.721910,1,1,2025-07-16 13:27:31.951622,2025-07-16 13:27:33.721920
2,מי דיווח על שימוש בטאוננקטל בטקסים אצטקים?,מוטיבים של פטריות נמצאו בהריסות של מקדש שבט המ...,1,0,13d055ef-8c68-42dc-ae21-fe2a99eaf3a1,מי דיווח על שימוש בטאוננקטל בטקסים אצטקים?,ברנרדינו דה סהגון,מוטיבים של פטריות נמצאו בהריסות של מקדש שבט המ...,30696fe6b2459161b8f70051b1fc73d6,13d055ef-8c68-42dc-ae21-fe2a99eaf3a1,...,377,50,gemini-2.0-flash-lite,1.451190,1.451209,2025-07-16 13:27:33.403154,2,1,2025-07-16 13:27:31.951923,2025-07-16 13:27:33.403163
3,מדוע חשופים מעשני קנאביס לריכוזים גבוהים של קר...,"עם זאת, יש ראיות לכך שמעשני קנאביס כבדים עלולי...",2,0,f79a980c-d52f-4629-abe4-322a8033ff93,מדוע חשופים מעשני קנאביס לריכוזים גבוהים של קר...,בעקבות השאיפות הארוכות ללא מסנן,"עם זאת, יש ראיות לכך שמעשני קנאביס כבדים עלולי...",68c786be386f4ccf1cb7f369b472eda4,f79a980c-d52f-4629-abe4-322a8033ff93,...,390,65,gemini-2.0-flash-lite,1.886128,1.886142,2025-07-16 13:27:33.838189,3,1,2025-07-16 13:27:31.952027,2025-07-16 13:27:33.838197
4,"באיזה בית חולים פסיכיאטרי עובד ד""ר מוטי משיח?",קיים מתאם מובהק בין שימוש בקנאביס לסיכוי של אד...,2,1,e56af8e0-48ec-4eb6-a9f1-fcd674a8aa18,"באיזה בית חולים פסיכיאטרי עובד ד""ר מוטי משיח?",אברבנאל,קיים מתאם מובהק בין שימוש בקנאביס לסיכוי של אד...,91df80e716f986f70e2d9df9ff9c850c,e56af8e0-48ec-4eb6-a9f1-fcd674a8aa18,...,520,52,gemini-2.0-flash-lite,1.463492,1.463509,2025-07-16 13:27:33.416904,4,1,2025-07-16 13:27:31.953372,2025-07-16 13:27:33.416914


In [62]:
df_queries.columns

Index(['anchor_text', 'positive_text', 'index', 'paragraph_index',
       'question_id', 'question', 'answer', 'context', 'context_hash', 'id',
       'text', 'context_id', 'context_text', 'dataset_name', 'tokenizer',
       'source_lang', 'target_lang', 'text_key', 'translation_key',
       'context_key', 'translation_query_key', 'translation_document_key',
       'system_prompt', 'user_prompt', 'prompt_prefix', 'prompt_file_name',
       'translation', 'input_tokens', 'output_tokens', 'model_name',
       'model_time', 'translation_time', 'timestamp', 'batch_idx',
       'batch_size', 'batch_datetime', 'translation_datetime'],
      dtype='object')

In [63]:
guid_to_context_target = dict(zip(df_documents2['context_hash'], df_documents2[f'context_{target_lang}']))
guid_to_context_source = dict(zip(df_documents2['context_hash'], df_documents2[f'context_{source_lang}']))

df_queries2 = df_queries.copy()
df_queries2 = df_queries2.rename(columns={
    'text': f'question_{source_lang}', 
    'translation': f'question_{target_lang}'
})

df_queries2[f"context_{source_lang}"] = df_queries2["context_id"].map(guid_to_context_source)
df_queries2[f"context_{target_lang}"] = df_queries2["context_id"].map(guid_to_context_target)

df_queries2 = df_queries2[['id', 'context_id', f'question_{source_lang}', f'question_{target_lang}', f'context_{source_lang}', f'context_{target_lang}']]
df_queries2['_source'] = dataset_name
df_queries2.head()

,id,context_id,question_hebrew,question_english,context_hebrew,context_english,_source
0,425478ad-1fb3-4a1a-a100-230cc56e2ccf,dc04e43cf20ca319f608b77dffabe637,"מה אכלו הרשב""י ובנו במערה?",What did Rashbi and his son eat in the cave?,"יהודה בן גרים סיפר על דבריו, ובעקבות זאת נדון ...","Yehuda ben Gerim told of his words, and as a r...",heq
1,1139748b-3e29-4c9f-8ff8-180d2e734757,8f9843fd2f675169908bac717c401891,למה הודח רבן גמליאל מנשיאותו?,Why was Rabban Gamliel removed from his positi...,"בצעירותו נמנה רשב""י עם החכמים שהתקבצו ב'כרם בי...","In his youth, Rashbi was among the sages who g...",heq
2,13d055ef-8c68-42dc-ae21-fe2a99eaf3a1,30696fe6b2459161b8f70051b1fc73d6,מי דיווח על שימוש בטאוננקטל בטקסים אצטקים?,Who reported on the use of teonanácatl in Azte...,מוטיבים של פטריות נמצאו בהריסות של מקדש שבט המ...,Mushroom motifs were found in the ruins of a M...,heq
3,f79a980c-d52f-4629-abe4-322a8033ff93,68c786be386f4ccf1cb7f369b472eda4,מדוע חשופים מעשני קנאביס לריכוזים גבוהים של קר...,Why are cannabis smokers exposed to high conce...,"עם זאת, יש ראיות לכך שמעשני קנאביס כבדים עלולי...","However, there is evidence that heavy cannabis...",heq
4,e56af8e0-48ec-4eb6-a9f1-fcd674a8aa18,91df80e716f986f70e2d9df9ff9c850c,"באיזה בית חולים פסיכיאטרי עובד ד""ר מוטי משיח?",In which psychiatric hospital does Dr. Moti Ma...,קיים מתאם מובהק בין שימוש בקנאביס לסיכוי של אד...,There is a significant correlation between can...,heq


In [64]:
# Save the DataFrame to a JSON file
output_file = f'data/{dataset_name}/{split}/queries.jsonl'
Path(output_file).parent.mkdir(parents=True, exist_ok=True)
df_queries2.to_json(output_file, orient='records', lines=True, force_ascii=False)

In [65]:
output_file

'data/heq/validation/queries.jsonl'

In [17]:
df_queries.head()

,id,title,context,question,answers,context_hash,text,context_id,context_text,dataset_name,...,input_tokens,output_tokens,model_name,model_time,translation_time,timestamp,batch_idx,batch_size,batch_datetime,translation_datetime
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'...",5566f9c0998385b8a8a2c94aa64aa980,When did Beyonce start becoming popular?,5566f9c0998385b8a8a2c94aa64aa980,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,rajpurkar/squad_v2,...,246.0,36.0,gemini-2.0-flash-lite,2.587168,2.587183,2025-07-08 13:24:52.976154,0,1,2025-07-08 13:24:50.388942,2025-07-08 13:24:52.976160
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star...",5566f9c0998385b8a8a2c94aa64aa980,What areas did Beyonce compete in when she was...,5566f9c0998385b8a8a2c94aa64aa980,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,rajpurkar/squad_v2,...,251.0,46.0,gemini-2.0-flash-lite,2.492141,2.492155,2025-07-08 13:24:52.881498,1,1,2025-07-08 13:24:50.389315,2025-07-08 13:24:52.881503
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}",5566f9c0998385b8a8a2c94aa64aa980,When did Beyonce leave Destiny's Child and bec...,5566f9c0998385b8a8a2c94aa64aa980,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,rajpurkar/squad_v2,...,253.0,59.0,gemini-2.0-flash-lite,2.614930,2.614943,2025-07-08 13:24:53.082593,2,1,2025-07-08 13:24:50.467624,2025-07-08 13:24:53.082598
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [...",5566f9c0998385b8a8a2c94aa64aa980,In what city and state did Beyonce grow up?,5566f9c0998385b8a8a2c94aa64aa980,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,rajpurkar/squad_v2,...,251.0,40.0,gemini-2.0-flash-lite,2.492606,2.492623,2025-07-08 13:24:52.964039,3,1,2025-07-08 13:24:50.471388,2025-07-08 13:24:52.964047
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}",5566f9c0998385b8a8a2c94aa64aa980,In which decade did Beyonce become famous?,5566f9c0998385b8a8a2c94aa64aa980,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,rajpurkar/squad_v2,...,247.0,36.0,gemini-2.0-flash-lite,2.455187,2.455204,2025-07-08 13:24:52.928191,4,1,2025-07-08 13:24:50.472951,2025-07-08 13:24:52.928199


In [19]:
from tqdm import tqdm
from datasets import load_dataset

data_files = {
        "queries": {
            "test": f'data/retrieval/squad_v2/{split}/queries.jsonl'
        },
        "documents": {
            "test": f'data/retrieval/squad_v2/{split}/documents.jsonl'
        }
    }

query_text_field = "question_english"
query_context_field = "context_hebrew"
document_text_field = "context_hebrew"
document_source_field = "_source"
main_source = "documents.jsonl"

queries_dataset = load_dataset("json", data_files=data_files["queries"], split="test")
documents_dataset = load_dataset("json", data_files=data_files["documents"], split="test")

questions = [item[query_text_field] for item in tqdm(queries_dataset, desc="Loading queries")]
gold = [item[query_context_field] for item in tqdm(queries_dataset, desc="Loading gold documents")]  # list of lists

unique_contexts = set()
deduped_contexts = []
contexts0 = [item for item in tqdm(documents_dataset, desc="Loading documents")]
# Deduplicate contexts
contexts0.sort(key=lambda x: 0 if x[document_source_field] == main_source else 1)
for i, context in tqdm(enumerate(contexts0), desc="Deduplicating contexts"):
    if context["guid"] not in unique_contexts:
        unique_contexts.add(context["guid"])
        deduped_contexts.append(context)
contexts = [c[document_text_field] for c in deduped_contexts]
print(len(deduped_contexts), "unique contexts found.")

# Find how many gold contexts are in the documents
found_gold_contexts = [c for c in set(gold) if c in contexts]
print(f"Found {len(found_gold_contexts)} gold contexts in the documents ({len(set(gold))}).")

FileNotFoundError: Unable to find '/home/nlp/achimoa/workspace/hebrew_text_retrieval/data/retrieval/squad_v2/train/queries.jsonl'

In [67]:
data_files2 = {
        "queries": {
            "test": 'data/retrieval/squad_v2/validation/queries.jsonl'
        },
        "documents": {
            "test": 'data/retrieval/squad_v2/validation/documents2.jsonl'
        }
}    

queries_dataset2 = load_dataset("json", data_files=data_files2["queries"], split="test")
documents_dataset2 = load_dataset("json", data_files=data_files2["documents"], split="test")
len(set([item["context_hebrew"] for item in tqdm(documents_dataset2, desc="Loading documents") if item["text"] is not None]))

Loading documents: 100%|██████████| 276204/276204 [00:20<00:00, 13439.70it/s]


1205

In [73]:
documents_dataset2[125000]

{'id': None,
 'context_hash': None,
 'context_english': None,
 'context_hebrew': None,
 '_source': 'wikipedia_hebrew_AllOfNewHebrewWikipediaWithArticles-Oct29-2023.forgpt.jsonl',
 'guid': 'ab2b029a96889c5149c7943a8e8452e5b206fcbaa2c401c5b8535f2d7f9c0c7e',
 'text': 'טוקיו דום\nטוקיו דום\nטוקיו דום (ביפנית: <foreign>; בבורסה:9681) הוא אצטדיון בייסבול הממוקם ברובע בונקיו שבטוקיו. הוא מכיל כ-55,000 מושבים והוא אצטדיון הבייסבול המקורה הגדול בעולם.\nהאצטדיון נפתח ב-17 במרץ 1988. הוא נבנה באתר בו שכן ולודרום ואצטדיון "קורקואן". כמו קורקואן לפניו, האצטדיון מארח מדי שנה את Toei Superheroes, מופע גיבורי על עם מסורת ארוכת שנים.\nהטוקיו דום מכונה "הביצה הגדולה" או "הביצה הגדולה של טוקיו" הודות לצורת הכיפה של גג האצטדיון. הגג הוא קרום גמיש שצורתו נשמרת על ידי הפעלת לחץ אוויר מועט אך רציף מצדו הפנימי של האצטדיון.\nהטוקיו דום הוא ביתה של קבוצת הבייסבול יומיורו ג\'יינטס, אך הוא מארח גם אירועים אחרים: משחקי כדורסל, פוטבול, כדורגל, אירועי היאבקות מקצועית, אמנויות לחימה משולבות, קיקבוקסינג, ומרוצי Monste

In [ ]:
len(questions), len(gold), len(contexts)

(1156, 1156, 0)

In [37]:
queries_dataset[0]

{'id': '56ddde6b9a695914005b9629',
 'context_id': '4792c0101c483fd5cb5191f0d39d4426',
 'question_english': 'When were the Normans in Normandy?',
 'question_hebrew': 'מתי הנורמנים היו בנורמנדי?',
 'context_english': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'context_hebrew': 'הנורמנים (נורמנית: 

In [41]:
unique_contexts

set()